# Part One

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob
import copy 

### Pulling Info From YML File

In [2]:
# Localise with . files 
# config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'
config_filename = './ds41_42_config.yml'

In [3]:
config_filename

'./ds41_42_config.yml'

In [4]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [5]:
first_ID = config['inputs']['first_ID']

In [6]:
second_ID = config['inputs']['second_ID']

In [7]:
output_path = config['inputs']['output_path']

In [8]:
first_ID_datafiles = config[first_ID].values()

In [9]:
second_ID_datafiles = config[second_ID].values()

### Downloading SQL Data

In [10]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [11]:
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [12]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [13]:
# SQl Data at different levels
jur_level = sql_df_all[sql_df_all['geotype']=='jurisdiction'].drop('geotype', axis=1)
cpa_level = sql_df_all[sql_df_all['geotype']=='cpa'].drop('geotype', axis=1)

### Downloading the first_ID Data

In [ ]:
# first_ID Data Frame
first_ID_df = pd.DataFrame()
for file_name in first_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    first_ID_df = first_ID_df.append(working_df)

In [ ]:
# Save the features_first_ID for future use (Used when creating the diff file)
features_first_ID = first_ID_df.drop(['mgra', 'year'], axis=1).columns

In [ ]:
comparison_first_ID_no_geozone = copy.deepcopy(first_ID_df)

In [ ]:
# Adding SQl Data to first_id_df
comparison_first_ID_processed_data = comparison_first_ID_no_geozone.merge(jur_level.drop_duplicates(), how='left', on='mgra')

In [ ]:
# making it original
comparison_first_ID_processed_data.columns = [x + f'_{first_ID}' for x in comparison_first_ID_processed_data.columns]

In [ ]:
comparison_first_ID_processed_data.head()

In [ ]:
#comparison_first_ID_processed_data.to_csv(f"{output_path}/comparison_{first_ID}_processed_data.csv")

### Downloading  the second_ID Data

In [ ]:
# second_id Data Frame
second_ID_df = pd.DataFrame()
for file_name in second_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    second_ID_df = second_ID_df.append(working_df)

In [ ]:
# Save the features of the second ID for future use (Used when creating the diff file)
features_second_ID = second_ID_df.drop(['mgra', 'year'], axis=1).columns

In [ ]:
comparison_second_ID_no_geozone = copy.deepcopy(second_ID_df)

In [ ]:
# Adding SQl Data to second_id_df
comparison_second_ID_processed_data = comparison_second_ID_no_geozone.merge(jur_level.drop_duplicates(), how='left', on='mgra')

In [ ]:
# making it original
comparison_second_ID_processed_data.columns = [x + f'_{second_ID}' for x in comparison_second_ID_processed_data.columns]

In [ ]:
comparison_second_ID_processed_data.head()

In [ ]:
# comparison_second_ID_processed_data.to_csv(f"{output_path}/comparison_{second_ID}_processed_data.csv")

### Concatenate both DS dataframes

In [ ]:
# Added geozone to merge keys to account for mgra's in multiple jurisdictions (or other geographical levels)
first_second_ID_comparison = comparison_first_ID_processed_data.merge(
    comparison_second_ID_processed_data,
    how='left',
    left_on=[f'mgra_{first_ID}',
             f'year_{first_ID}',
             f'geozone_{first_ID}'],
    right_on=[f'mgra_{second_ID}',
             f'year_{second_ID}',
             f'geozone_{second_ID}'])

In [ ]:
# Clean green combined
first_second_ID_comparison = first_second_ID_comparison.drop([f'mgra_{second_ID}', f'year_{second_ID}', f'geozone_{second_ID}'], axis=1)
first_second_ID_comparison = first_second_ID_comparison.rename(columns={f'mgra_{first_ID}': 'mgra', f'year_{first_ID}': 'year', f'geozone_{first_ID}': 'geozone'})

In [ ]:
# Because we're summing, if using series 14 data, mgra's in multiple jurisdictions will be counted multiple times
first_second_ID_comparison = first_second_ID_comparison.groupby(['mgra', 'year']).sum()

In [ ]:
first_second_ID_comparison.head()

In [ ]:
#first_second_ID_comparison.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison")

### Creating the complete Diff File

In [ ]:
# Finding features common to both DSID data frames
list1_as_set = set(features_first_ID)
intersection = list1_as_set.intersection(features_second_ID)

shared_features = list(intersection)

In [ ]:
# Calculate diff values between the two DS_ID's
diff_df = pd.DataFrame()
for column in shared_features:
    diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']

In [ ]:
diff_df.head()

In [ ]:
#diff_df.to_csv(f"{output_path}/{first_ID}_{second_ID}_complete_diff")

In [ ]:
# Check CPA dupes also ( where one mgra in multiple cpa )

## CPA level Data

In [ ]:
# Adding SQl Data to first_second_ID_combined dataframe
comparison_processed_data_cpa = first_second_ID_combined.merge(cpa_level, how='left', on='mgra')

In [ ]:
comparison_first_ID_no_geozone

### first_ID df

In [ ]:
# Adding SQl Data to first_id_df
comparison_first_ID_processed_data_cpa = comparison_first_ID_no_geozone.merge(cpa_level, how='left', on='mgra')

In [ ]:
# Remove mgra column
comparison_first_ID_processed_data_cpa = comparison_first_ID_processed_data_cpa.drop('mgra', axis=1)

In [ ]:
# Aggregate values by CPA and year (Included code to see "*Not in a CPA*" values)
comparison_first_ID_processed_data_cpa.groupby(['geozone', 'year']).sum()#.drop('*Not in a CPA*')

In [ ]:
# Remove mgra column
comparison_processed_data_cpa = comparison_processed_data_cpa.drop('mgra', axis=1)

In [ ]:
# Aggregate values by CPA and year (Included code to see "*Not in a CPA*" values)
comparison_processed_data_cpa = comparison_processed_data_cpa.groupby(['geozone', 'year']).sum().drop('*Not in a CPA*')

In [ ]:
comparison_processed_data_cpa.head()

In [ ]:
#comparison_processed_data_cpa.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_cpa")

## Jurisdiction level Data

In [ ]:
# Adding SQl Data to first_second_ID_combined dataframe
comparison_processed_data_jur = first_second_ID_combined.merge(jur_level, how='left', on='mgra')

In [ ]:
# Remove mgra column
comparison_processed_data_jur = comparison_processed_data_jur.drop('mgra', axis=1)

In [ ]:
# Aggregate values by Jurisdiction and year (Included code to see "*Not in a CPA*" values)
comparison_processed_data_jur = comparison_processed_data_jur.groupby(['geozone', 'year']).sum()

In [ ]:
comparison_processed_data_jur.head()

In [ ]:
#comparison_processed_data_jur.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_jur")

In [ ]:
# Unsure about file name, maybe spell out jurisdiction instead of jur?

In [ ]:
# Should we rename geozone column to CPA or jurisdiction?

## Region level Data

## Individual Level?

# Brainstorming for Part 2

In [ ]:
config['dof']['dof_data']

In [ ]:
first_second_ID_comparison.head()

In [ ]:
# MGRA level stats
first_second_ID_comparison.describe(percentiles=[0.5])